In [1]:
# Load Real World Dataset
import pandas as pd # Import the pandas library
import os


data = pd.read_csv('/content/cleaned_data (1).csv')
data.isnull().sum()#Jika nilai null 0 tidak perlu lagi data cleaning dan

df = pd.DataFrame(data)
df


100%|██████████| 20.0M/20.0M [00:00<00:00, 64.8MB/s]

Extracting files...


,fullAddress,postcode,country,outcode,latitude,longitude,bathrooms,bedrooms,floorAreaSqM,livingRooms,...,saleEstimate_upperPrice,saleEstimate_confidenceLevel,saleEstimate_ingestedAt,saleEstimate_valueChange.numericChange,saleEstimate_valueChange.percentageChange,saleEstimate_valueChange.saleDate,history_date,history_price,history_percentageChange,history_numericChange
0,"Flat 1, White Rose Court, Widegate Street, Lon...",E1 7ES,England,E1,51.517972,-0.078028,2.0,2.0,73.0,1.0,...,684000.0,HIGH,2024-10-07T13:26:59.894Z,28000.0,4.494382,2023-10-31,2023-10-31,623000.0,3.833333,23000.0
1,"Flat 1, White Rose Court, Widegate Street, Lon...",E1 7ES,England,E1,51.517972,-0.078028,2.0,2.0,73.0,1.0,...,684000.0,HIGH,2024-10-07T13:26:59.894Z,28000.0,4.494382,2023-10-31,2016-03-23,600000.0,155.319149,365000.0
2,"Flat 1, White Rose Court, Widegate Street, Lon...",E1 7ES,England,E1,51.517972,-0.078028,2.0,2.0,73.0,1.0,...,684000.0,HIGH,2024-10-07T13:26:59.894Z,28000.0,4.494382,2023-10-31,2005-11-28,235000.0,181.437126,151500.0
3,"Flat 4, White Rose Court, Widegate Street, Lon...",E1 7ES,England,E1,51.517972,-0.078028,1.0,1.0,51.0,1.0,...,624000.0,MEDIUM,2024-10-07T13:26:59.894Z,182000.0,47.272727,2014-09-30,2014-09-30,385000.0,54.000000,135000.0
4,"Flat 5, White Rose Court, Widegate Street, Lon...",E1 7ES,England,E1,51.517972,-0.078028,1.0,2.0,50.0,1.0,...,642000.0,HIGH,2024-10-07T13:26:59.894Z,36000.0,6.260870,2023-11-14,2023-11-14,575000.0,64.285714,225000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28911,"6 Lismore Road, London, N17 6LE",N17 6LE,England,N17,51.589626,-0.084707,1.0,2.0,96.0,2.0,...,766000.0,HIGH,2024-10-07T13:26:59.894Z,60000.0,8.955224,2021-04-30,2004-08-12,91250.0,133.974359,52250.0
28912,"4 Lordship Lane, London, N17 8NA",N17 8NA,England,N17,51.598752,-0.069278,1.0,3.0,95.0,2.0,...,583000.0,MEDIUM,2024-10-07T13:26:59.894Z,160000.0,43.243243,2019-03-15,2019-03-15,370000.0,105.555556,190000.0
28913,"57 Lordship Lane, London, N17 6RU",N17 6RU,England,N17,51.598592,-0.072129,1.0,3.0,127.0,1.0,...,670000.0,MEDIUM,2024-10-07T13:26:59.894Z,184000.0,43.294118,2022-07-28,2022-07-28,425000.0,92.307692,204000.0
28914,"160 Lordship Lane, London, N17 7QS",N17 7QS,England,N17,51.598187,-0.080853,2.0,3.0,89.0,1.0,...,510000.0,MEDIUM,2024-10-07T13:26:59.894Z,38000.0,8.941176,2016-09-02,2016-09-02,425000.0,156.797583,259500.0


In [3]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col

# Step 1: Initialize SparkSession
spark = SparkSession.builder.appName("Classification").getOrCreate()

# Step 2: Load Dataset
file_path = '/content/cleaned_data (1).csv'
data = spark.read.csv(file_path, header=True, inferSchema=True)

data = data.withColumn("saleEstimate_upperPrice", col("saleEstimate_upperPrice").cast("double"))
data = data.withColumn("history_price", col("history_price").cast("double"))
data = data.withColumn("bedrooms", col("bedrooms").cast("double"))  # Pastikan 'bedrooms' juga numerik
data = data.withColumn("longitude", col("longitude").cast("double"))

# Step 3: Data Preprocessing
# Check for null values
# Rename columns with periods (.) to avoid issues
for col in data.columns:
    if "." in col:
        new_col = col.replace(".", "_")  # Replace periods with underscores
        data = data.withColumnRenamed(col, new_col)

data = data.dropna()  # Drop rows with missing values

# Select features and target column
# Assuming 'Air Quality' is the column to classify; adjust it based on your dataset
features_columns = ['saleEstimate_upperPrice', 'history_price', 'bedrooms', 'longitude']
assembler = VectorAssembler(inputCols=features_columns, outputCol="features")

# Transform the dataset
data = assembler.transform(data)

# Convert target column to numeric
# Change 'target' to 'Air Quality' since it is the target column
indexer = StringIndexer(inputCol="saleEstimate_confidenceLevel", outputCol="label")
data = indexer.fit(data).transform(data)

# Split into training and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Step 4: Build Classification Model
rf_classifier = RandomForestClassifier(featuresCol="features", labelCol="label")

# Step 5: Hyperparameter Tuning using Cross-Validation
# Define the parameter grid to search
param_grid = ParamGridBuilder() \
    .addGrid(rf_classifier.numTrees, [50, 100, 150]) \
    .addGrid(rf_classifier.maxDepth, [5, 10, 15]) \
    .build()

# Define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Create CrossValidator
cross_validator = CrossValidator(estimator=rf_classifier,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=3)  # Use 3 folds for cross-validation

# Fit the model using CrossValidator
cv_model = cross_validator.fit(train_data)

# Get the best model
best_model = cv_model.bestModel

# Step 6: Evaluate the Best Model
predictions = best_model.transform(test_data)

# Calculate accuracy
accuracy = evaluator.evaluate(predictions)

# Print evaluation metrics
print(f"Accuracy of Best Model: {accuracy:.2f}")

# Optional: Other metrics (precision, recall, F1-score)
# ... (same as before)

# Stop SparkSession
spark.stop()

Accuracy of Best Model: 0.82
